In [1]:
# 시간이 오래 걸리므로 따로 만든다.
from pororo import Pororo
from tqdm.auto import tqdm
from datasets import load_metric, load_from_disk, Dataset, DatasetDict

ner = Pororo(task="ner", lang="ko")


class Question_tag():
    def __init__(self,datasets):
        datasets = datasets.to_pandas()
        datasets['Q_tag'] = 0
        self.dataset = datasets
    
    def who(self):
        
        who_endpoint = ['사람','인물', '당사자','투수','직업','지위','봔관',
                        '선임자','여자','학자'
               ,'직위','형','부친','선수',
                '자는','지휘자','직책','아버지','문학가','팀','왕','마을'
               ,'후손','남편','손자','부족','상대','언론사','신분','직분',
                '계급','관직','대상','지도자'
               ,'진행자','원작자','자식','전함','동생','친구','화자','환자',"기업","이는"]
        who_contain = ['누가','누구',"어떤 인물", "어떤 사람","사람의 이름"]
        # Q_tag가 안 들어간 것(0인 값만) 가져와서 써야한다!
        tag_index = self.dataset.loc[self.dataset['Q_tag']==0].index
        
        for idx in tqdm(tag_index):
            query = train[idx]['question']
            query_split = query.split()
            check=False
            
            
            for who_is in who_contain:
                if who_is in query:
                    self.dataset['Q_tag'][idx]="[WHO]"
                    check = True
                    break
            if check:
                continue
            
            for who_word in who_endpoint:
                if who_word in query_split[-1]:
                    self.dataset['Q_tag'][idx]="[WHO]"
                    check = True
                    break
            if check:
                continue
            
            # 진행자 이름, 환자 이름 등등 포함 => 이유는 사물 이름은? 이라는 경우때문에 사용
            if "이름" in query_split[-1]:
                for who_word in who_endpoint:
                    if who_word in query_split[-2]:
                        self.dataset['Q_tag'][idx]="[WHO]"
                        check=True
                        break
            if check:
                continue
            
            # endpoint list를 새로운 단어 나올때마다, 계속 추가할 수는 없음..
            # 그래서 endpoint에 나오는 단어가 ner에 속하는 부분을 적음
            # ex. 화자 -> CIVILIZATION, 환자 -> OCCUPATION
            if ner(query_split[-1])[0][1]=="CIVILIZATION" or ner(query_split[-1])[0][1]=="OCCUPATION":
                self.dataset['Q_tag'][idx]="[WHO]"
    
    # 누구 때문인가? 이런 질문들 때문에 why가 앞에 나온다.
    def why(self):
        why_endpoint = ['때문']
        why_contain = [' 왜 ', '원인', '요인', '계기','이유','무엇때문에']
        tag_index = self.dataset.loc[self.dataset['Q_tag']==0].index
        
        for idx in tqdm(tag_index):
            query = train[idx]['question']
            query_split = query.split()
            check=False
            
            for why in why_endpoint:
                if why in query_split[-1]:
                    self.dataset['Q_tag'][idx]="[WHY]"
                    check=True
                    break
            if check==True:
                continue
            
            for why in why_contain:
                if why in query:
                    #'보라메이유'라는 사람이 있기에 '~ 이유때문에'라는 경우를 살리기 위해 적용
                    if why=="이유" and query[query.find(why)-1]==' ':
                        self.dataset['Q_tag'][idx]="[WHY]"
                        check=True
                        break
                    elif why!="이유":
                        self.dataset['Q_tag'][idx]="[WHY]"
                        check=True
                        break
    
    def where(self):
        where_endpoint = ["어디","곳은","지역","학교","국가","나라","군대","장소","땅은","근무지","국가"
                 ,"도시","패전국","공간","국적","이동","부대","학교","출신","마을","위치","소재지","지역"
                         ,"거주지","도착지","목적지"]
        where_contain = ["어느 나라","어느 도시","어느 국가", "어느 방향","어느 주",
                         "어디서","어디에서","어디까지","어디를","어디로","어디에"]
        
        tag_index = self.dataset.loc[self.dataset['Q_tag']==0].index
        
        for idx in tqdm(tag_index):
            query = train[idx]['question']
            query_split = query.split()
            check = False
            
            for wh in where_endpoint:
                if wh in query_split[-1]:
                    self.dataset['Q_tag'][idx]="[WHERE]"
                    check=True
                    break
            if check:
                continue
            
            for wh in where_endpoint:
                d = "어느 "+wh
                if d in query:
                    self.dataset['Q_tag'][idx]="[WHERE]"
                    check=True
                    break
            if check:
                continue
            
            for wh in where_contain:
                if wh in query:
                    self.dataset['Q_tag'][idx]="[WHERE]"
                    check=True
                    break
    
    def how(self):
        how_contain = ["어떻게", "어느 정도", "몇 명","어떤 방식","어떤 방법",
                       "나이 차이","몇 개", "어떠한 공격 방식","얼마의 비중"]
        how_endpoint_equals = ["수는","양은"]
        how_endpoint = ["방법","개수","갯수","나이","방식","액수","금액","거리","온도","얼마","수는","양은","인구"
                        ,"길이"]
        
        tag_index = self.dataset.loc[self.dataset['Q_tag']==0].index
        
        for idx in tqdm(tag_index):
            query = train[idx]['question']
            query_split = query.split()
            check=False
            
            for how in how_endpoint_equals:
                if how==query_split[-1]:
                    self.dataset['Q_tag'][idx]="[HOW]"
                    check=True
                    break
            
            if check:
                continue
            
            for how in how_endpoint:
                # 얼마만이면 -> when!
                if how in query_split[-1] and how=="얼마" and "얼마만" not in query_split[-1]:
                    self.dataset['Q_tag'][idx]="[HOW]"
                    check=True
                    break
                    
                elif how in query_split[-1] and how!="얼마":
                    self.dataset['Q_tag'][idx]="[HOW]"
                    check=True
                    break
            if check:
                continue
            
            for how in how_contain:
                if how in query:
                    self.dataset['Q_tag'][idx]="[HOW]"
                    check=True
                    break
    
    def when(self):
        when_endpoint = ["년도", "연도", "날짜","시기","며칠","기간","언제","몇년","요일","계절","해는",
                         "날은","때는","달은","시대","시간"]
        when_endpoint_equals = ["해는","날은","때는","달은"]
        when_contain = ["몇 월","사용기간","언제부터","언제라고","몇 년","몇 기","언제","어느 계절"
               "언제부터","언제까지","언제",'어느 계절','얼마 뒤','얼마나 늦게', '얼마나 빨리', 
                '얼마나 오래', '얼마만에', '얼마동안','어느 해', '어느 요일', 
                '어느 시대', '어느 시기', '얼마 후','몇 시간', '몇시간', '몇 분', '몇분',
                '얼마 이내', '얼마나 걸렸']
        tag_index = self.dataset.loc[self.dataset['Q_tag']==0].index
        
        for idx in tqdm(tag_index):
            query = train[idx]['question']
            query_split = query.split()
            check=False
            
            for when in when_endpoint:
                if when in query_split[-1]:
                    self.dataset['Q_tag'][idx]="[WHEN]"
                    check=True
                    break
                    
            if check==True:
                continue
            
            for when in when_endpoint_equals:
                if when==query_split[-1]:
                    self.dataset['Q_tag'][idx]="[WHEN]"
                    check=True
                    break
            if check==True:
                continue
            
            for when in when_contain:
                if when in query:
                    self.dataset['Q_tag'][idx]="[WHEN]"
                    check=True
                    break
    
    def what(self):
        tag_index = self.dataset.loc[self.dataset['Q_tag']==0].index
        self.dataset['Q_tag'][tag_index]="[WHAT]"
        
    def Query_tagging(self):
        self.who()
        self.where()
        self.why()
        self.how()
        self.when()
        self.what()
        return Dataset.from_pandas(self.dataset)

2021-10-31 15:16:41.584139: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[Korean Sentence Splitter]: Initializing Kss...


In [2]:
from datasets import load_from_disk
import re

train = load_from_disk('/opt/ml/data/train_dataset/train')
train

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})

In [3]:
valid = load_from_disk('/opt/ml/data/train_dataset/validation')
valid

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 240
})

In [4]:
test = load_from_disk('/opt/ml/data/test_dataset')
test

DatasetDict({
    validation: Dataset({
        features: ['id', 'question'],
        num_rows: 600
    })
})

In [ ]:
tags = Question_tag(train)
tags_complete=tags.Query_tagging()
tags_complete.to_pandas().to_csv('/opt/ml/question_tag_trainset.csv')

In [ ]:
tags = Question_tag(valid)
tags_complete=tags.Query_tagging()
tags_complete.to_pandas().to_csv('/opt/ml/question_tag_validset.csv')

In [ ]:
tags = Question_tag(test)
tags_complete=tags.Query_tagging()
tags_complete.to_pandas().to_csv('/opt/ml/question_tag_testset.csv')